# HW 1 - Patryk Słowakiewcz

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import axes as ax
import seaborn as sns
from scipy import stats
import datetime
import plotly.graph_objs as go
import plotly.express as px
import pandas_profiling

## Wczytanie i wstępna eksploracja

In [ ]:
df = pd.read_csv("forest_fires_dataset.csv")

In [ ]:
df.head(10)

In [ ]:
#sprawdzam braki w danych
df.info()

In [ ]:
# zamieniamy dni i miesiące na liczby

#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#df['month_int'] = le.fit_transform(df['month']) + 1
#df['day_int'] = le.fit_transform(df['day']) + 1

# ale to niestety nie działa bo są nie pokolei

## Zmiana dni i miesięcy na liczby

In [ ]:
def day_change(weekday):
    if weekday == 'mon':
        return 1
    if weekday == 'tue':
        return 2
    if weekday == 'wed':
        return 3
    if weekday == 'thu':
        return 4
    if weekday == 'fri':
        return 5
    if weekday == 'sat':
        return 6
    if weekday == 'sun':
        return 7

In [ ]:
m_list = [datetime.datetime.strptime(i, '%b') for i in df['month']]
df['month_int'] = [i.month for i in m_list]
df['day_int'] = [day_change(i) for i in df['day']]

In [ ]:
df.describe()

## Badanie targetu (area)

In [ ]:
#Spardzam rozkład naszego y którym jest 'area'
sns.boxplot(df['area'])
# widzimy bardzo duże odstępstwa od mediany na prawo (jest to spowodowane dużą ilością zartości 0 i bliskich)

In [ ]:
# po usunięciu wartości 0 i zastosowaniu skali logarytmicznej otrzymujemy rozkład bliski rozkładowi normalnemu
cdf2 = np.isclose(df['area'], 0)
cdf = np.logical_not(cdf2)
sns.distplot(np.log(df[cdf]['area']))

In [ ]:
#dodajemy kolumnę z wartościamu log(df('area')) aby łatwiej ją było analizować ( dla 0 jest None)
l = [np.log(i) if i != 0 else None for i in df['area']]
df['area_log'] = l

### Histogramy dla każdej zmiennej przy czym bierzemy obserwacje jedynie dla przypadków pożarów.

Widzimy z poniższych wykresów, że ['DC', 'DMC', 'FFMC', 'ISI', 'RH', 'temp', 'wind'], mogą być w pewien sposób zależne co wynika również ze strony kanadyjskiego leśnictwa poniważ wskażniki te są wyliczane na podstawie warunków atmosferycznych (https://cwfis.cfs.nrcan.gc.ca/background/summary/fwi). 

In [ ]:
df[cdf].hist(bins= 20,figsize=(18,12))
plt.show()

### Pairplot
Sprawdzamy możliwe korelacje pomiędzy wielkością pożarów a innymi zmiennymi.

Nie widać, żadnych liniowych zależności ale można wyciągnąć wnioski, że pożary pojawiają się kiedy ilość opdaów jest zerowa co jest dość intuicyjne.

In [ ]:
sns.pairplot(df[cdf], y_vars="area_log", x_vars=df.columns.values[4:8])
sns.pairplot(df[cdf], y_vars="area_log", x_vars=df.columns.values[8:12])
sns.pairplot(df[cdf], y_vars="area_log", x_vars=df.columns.values[13:15])

plt.show()

### Heatmapa korelacji

Widzimy, że podane w ramce wskaźniki są ze sobą trochę skorelowane (w szczególności DC oraz DMC) ciekawa jest również wysoka korelacja DC z miesiącami pomimo, że data nie ma bezpośredniego wpływu na wyliczenia DC

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True)

plt.show()

### Umiejscownie pożarów
Sprawdzimy jak rozkładaja się średnia pożarów z zależności od zmiennych X oraz Y

In [ ]:
# 
fire_matrix = np.zeros((10,11))
for i in range(1,len(df)):
    k = df['area'][i]
    if k != 0:
        k = np.log(k)
    fire_matrix[df.iloc[i, 1], df.iloc[i, 0]] += k

In [ ]:
freq_matrix = np.zeros((10,11))
for i in range(1,len(df)):
    k = df['area'][i]
    if k != 0:
        freq_matrix[df.iloc[i, 1], df.iloc[i, 0]] += 1

In [ ]:
sns.heatmap((fire_matrix/freq_matrix), cmap='hot_r')

Widać pewną reguralność wzdłuż na powyższej 'mapie'. Pożary rozkładają się wzdłuż Y = 4

## Rozkład średniej temperatury według miesięcy wraz z odchyleniem standardowm. Poniżej widać, wzrost w miesiądach letnich co jest spodziewaną zależnością

In [ ]:
months = df.groupby(['month_int']).mean()
months_std =  df.groupby(['month_int']).std()
months_std.loc[11] = 0
months_max =  df.groupby(['month_int']).max()
months_min =  df.groupby(['month_int']).min()

In [ ]:
x = months.index
y = months['temp']
y_upper = y + months_std['temp']
y_lower = y - months_std['temp']

plt.figure(figsize=(12,8))
plt.plot(x, y)
plt.fill_between(x, y_upper, y_lower, alpha = 0.1)
plt.title("Mean temp with std")
plt.xlabel("Months")
plt.ylabel('Temperature')
plt.show()

In [ ]:
what = 'DMC'
x = months.index
y = months[what]
y_upper = y + months_std[what]
y_lower = y - months_std[what]

plt.figure(figsize=(12,8))
plt.plot(x, y)
plt.fill_between(x, y_upper, y_lower, alpha = 0.1)

plt.show()

Badając te dwie zmienne można zauważyć, że obie rosną w okresie letnim. Warto zatem zbadać jak inne zmienne zachowują się na przestrzenii roku

## Pairploty ze zwzględu na miesiące

In [ ]:
sns.pairplot(df, y_vars=["month_int"], x_vars=df.columns.values[4:8])
sns.pairplot(df, y_vars='month_int', x_vars=df.columns.values[8:12])
sns.pairplot(df, y_vars='month_int', x_vars=df.columns.values[[12,15]])


Na powyższych wykresach widać jak zmienne DMC oraz temp zwiększają wartość w okresie letnim ale to już zaobserwowaliśmy na wcześniejszych wykresach. Ale dzięki tym pairplotom widzimy, że również DC oraz ISI są mocno zależne od miesiąca.

## Python-profilig
Jest to dobre i wygodne narzędzie do szybkiej eksploracji danych dzięki któremu możemy zbadać rozkłady poszczególnych zmiennych. Dzięki heatmapie korelacji można też zauważyć które zmienne warto odrzucić. 
Wadami które mogą sprawiać problemy to brak możliwości wyboru jak gęste mają być histogramy pownieważ w pewnych przypadkach uniemożliwia to dopasowanie odpowiedniego rozkładu. Badane są wszystkie dane więc ręcznie trzeba wykonywać pewne przekształcenia np. zmienna 'area' jest mocno skoncentrowana przy 0 więc bez przeksztalceń nie możliwe jest zaobserwowanie jej właściwego rozkładu.
Tak samo korelacja wyłapuje jedynie liniowe zależności, więc jeżeli chcemy badać bardziej złożone zależności trzeba to robić ręcznie.


In [ ]:
df.profile_report()